In [ ]:
import random
import dill
from game import Game, Move, Player
from strategies.rl import Q_learing, CustomState,  get_coordinates

class RandomPlayer(Player):
    def __init__(self) -> None:
        super().__init__()

    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
        from_pos = (random.randint(0, 4), random.randint(0, 4))
        move = random.choice([Move.TOP, Move.BOTTOM, Move.LEFT, Move.RIGHT])
        return from_pos, move


class MyPlayer(Player):
    def __init__(self) -> None:
        super().__init__()

    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
        from_pos = (random.randint(0, 4), random.randint(0, 4))
        move = random.choice([Move.TOP, Move.BOTTOM, Move.LEFT, Move.RIGHT])
        return from_pos, move
    
class RLPlayer(Player):
    def __init__(self, learning_rate, discount_factor, pretrain_path=None, save_model_path=None, max_steps=None) -> None:
        super().__init__()
        ql = Q_learing(learning_rate, discount_factor, pretrain_path=pretrain_path, max_steps=max_steps)
        steps, self.value_dictionary = ql.train()

        if save_model_path is not None:
            d = {'steps': steps, 'value_dictionary': self.value_dictionary}

            with open(save_model_path, 'wb') as outfile:
                dill.dump(d, outfile)
        
    def make_move(self, game: Game) -> tuple[tuple[int, int], Move]:
        current_state = CustomState(get_coordinates(game.get_board()))
        list_action = sorted(self.value_dictionary[current_state], key=self.value_dictionary[current_state].get)
        '''
        for action in list_action:
            if action not in (current_state.x.union(current_state.o)):
                current_state.x.add(action)
                break
        '''

        from_pos, move = list_action[0]
        print(from_pos, move)
        return from_pos, move
    
g = Game()

player1 = RLPlayer(0.1, 0.7, pretrain_path='train_results/rl.pik', save_model_path='train_results/rl.pik', max_steps=50)
player2 = RandomPlayer()



In [ ]:
import dill
from strategies.rl import CustomState, print_dictionary

with open('train_results/rl.pik', 'rb') as f:
    my_dict = dill.load(f)

print(my_dict)
print(my_dict['steps'])
print_dictionary(my_dict['value_dictionary'])
print(len(my_dict['value_dictionary']))

In [ ]:
g.print()
winner = g.play(player1, player2)
g.print()
print(f"Winner: Player {winner}")